### Building the neural network model

In [7]:
## Defining the model

from keras.models import Sequential
from keras.layers import Dense

def build_model(params, input_shape):
    model = Sequential()
    model.add(Dense(params['n_units_layer_0'], activation='relu', input_shape=input_shape))
    if 'n_units_layer_1' in params:  # Check if the key exists before adding the layer
        model.add(Dense(params['n_units_layer_1'], activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model



### Hyperparameter tuning with Baysean optimization

In [ ]:
## Defining the objective function and search space

from hyperopt import fmin, tpe, hp
from keras.callbacks import EarlyStopping

# Hyperparameter Space
space = {
    'n_units_layer_0': hp.choice('n_units_layer_0', [32, 64, 128]),
    'n_layers': hp.choice('n_layers', [1, 2]),
    'n_units_layer_1': hp.choice('n_units_layer_1', [32, 64, 128]), # Added as a separate option
    'epochs': hp.choice('epochs', [50, 100, 150]),
    'batch_size': hp.choice('batch_size', [32, 64, 128]),
}

# Objective Function
def objective(params):
    if params['n_layers'] == 1:
        params.pop('n_units_layer_1') # Remove if n_layers == 1
        
    model = build_model(params, input_shape=(X_train_scaled.shape[1],))
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=params['epochs'], batch_size=params['batch_size'], callbacks=[early_stop], verbose=0)
    loss = history.history['val_loss'][-1]
    return loss

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=30, verbose=2)
print("Best hyperparameters found: ", best)